In [13]:
import pandas as pd
import PyPDF2
import re
from datetime import datetime

In [14]:
pdf = r'C:\Users\Carlo\OneDrive\Documents\_Report Output_CondensedStandardTimeDetailReport_89a86040-ec7e-45a8-b609-e8b142d86054_2025-02-03T07_56_41.696.pdf'
with open(pdf, 'rb') as file: # 'rb' is used to open binary files as world, pdf, excel, etc.
    reader = PyPDF2.PdfReader(file) # this become the doc in a object that could be read.

    payroll = []

    for num in range(len(reader.pages)): # '.pages' returns the number of pages in the doc. And len() returns how big is the list. 
        page = reader.pages[num]
        text = page.extract_text() # this extract the text that is in the page
        payroll.append(text)

print(payroll[0])

Condensed Employee Time Detail
Time 
Period :Previous Pay Period Executed 
on :2/03/2025 7:56 AM
Query : BWD JYC Temp 
AgencyPrinted for : Wheeler, Kathleen
Currency 
Code :USD
Page 1 of 9Employee: Aviles, Fabian ID: 104410 Time Zone:(GMT -05:00) Eastern Time (USA; 
Canada)
Status: Active Status Date: 1/20/25 Pay Rule: BWD JYC Temp Agency
Primary Job: Start Date: End Date: Primary Labor Category:
International Wire Group/Omega Wire Inc./
Williamstown/ Direct/Strand/WIL-CABDOU/WIL 
CABDOU L1,W4SB1/20/25   
Apply Date Pay Code In In Excep. Out Out Excep. Hours Wages Days
Job(Labor) Transfer
Comments Work Rule
2025-01-27  2:59 PM  12:00 AM  9.00  0.00
2025-01-28  2:53 PM Early In 12:00 AM  9.10  0.00
2025-01-29  2:53 PM Early In 12:00 AM  9.10  0.00
2025-01-30  2:54 PM  12:00 AM  9.00  0.00
2025-01-31  2:54 PM  12:00 AM  9.00  0.00
45.20 0.00 0.00
Job Summary
Pay Code Hours Wages Days
International Wire Group/Omega Wire Inc./Williamstown/Direct/Strand/WIL-CABDOU/WIL CABDOU L1,W4SB CC:N/A


In [15]:
employee = []
plant = []
regularTime = []
overTime = []

# time recording
monday = []
tuesday = []
wednesday = []
thursday = []
friday = []
saturday = []
sunday = []


def pageIteration(payroll):
    employee.clear()
    plant.clear()
    regularTime.clear()
    overTime.clear()
    monday.clear()
    tuesday.clear()
    wednesday.clear()
    thursday.clear()
    friday.clear()
    saturday.clear()
    sunday.clear()

    for page in payroll:
        extractEmployeeNameListData(page)

def extractEmployeeNameListData(page):
    
    name = nameExtraction(page)
    employee.append(name)
    
    place = plantExtraction(page)
    plant.append(place)

    timeRegular = regularTimeExtraction(page)
    regularTime.append(timeRegular)

    timeOver = overTimeExtraction(page)
    overTime.append(timeOver)

    timePerDayExtraction(page)


def nameExtraction(page):
    firstCoordinate = re.search('Employee:',page).span()[1]
    secondCoordinate = re.search('ID:',page).span()[0]
    fullname = page[firstCoordinate:secondCoordinate].strip().split(",")
    firstName = fullname[1]
    lastName = fullname[0]
    return (firstName+" "+lastName).strip()

def plantExtraction(page):
    firstCoordinate = re.search('International Wire Group/Omega Wire Inc./',page).span()[1]
    secondCoordinate = page.find('/',firstCoordinate)
    plantName = page[firstCoordinate:secondCoordinate].replace('/',"").replace(' \nPlant 1',"").strip()
    return plantName

def regularTimeExtraction(page):
    firstCoordinate = re.search('Regular',page).span()[1]
    secondCoordinate = page.find(' 0.00',firstCoordinate)
    regularTime = page[firstCoordinate:secondCoordinate]
    return regularTime

def overTimeExtraction(page):
    firstCoordinate = re.search('Overtime',page).span()[1]
    secondCoordinate = page.find(' 0.00',firstCoordinate)
    regularTime = page[firstCoordinate:secondCoordinate]
    return regularTime

def timePerDayExtraction(page):
    pattern = r'(\d{4}-\d{2}-\d{2}).*?(\d+\.\d{2})\s+0.00' # regular expression to extract date and time
    dateAndTime = re.findall(pattern, page)

    for index in range(len(dateAndTime)):
        time = str(dateAndTime[index][1])
        date = str(dateAndTime[index][0])
        weekDay = datetime.strptime(date, "%Y-%m-%d")
        weekDay = weekDay.strftime("%A") # verify if which week day is the date

        if weekDay == "Monday":
            monday.append(time)
        elif weekDay == "Tuesday":
            tuesday.append(time)
        elif weekDay == "Wednesday":
            wednesday.append(time)
        elif weekDay == "Thursday":
            thursday.append(time)
        elif weekDay == "Friday":
            friday.append(time)
        elif weekDay == "Saturday":
            saturday.append(time)
        elif weekDay == "Sunday":
            sunday.append(time)

In [16]:
pageIteration(payroll)
print(employee, plant, monday, tuesday, wednesday, thursday, friday, regularTime, overTime, sep='\n\n')

['Fabian Aviles', 'Eduardo Barboza', 'Anthony Casesa', 'Ian Gonzalez', 'Carlos Lucio', 'Eric Ojeda', 'Daniel Tenorio', 'Hector Tirrado', 'Gabriel Vera']

['Williamstown', 'Canastota', 'Canastota', 'Camden', 'Camden', 'Camden', 'Williamstown', 'Williamstown', 'Canastota']

['9.00', '12.00', '12.00', '12.00', '9.00', '12.00', '12.00', '12.00', '12.00']

['9.10', '11.00', '11.00', '12.00', '9.00', '12.00', '12.00', '12.00', '11.00']

['9.10', '11.00', '11.00', '9.00', '9.00', '9.00', '9.00', '9.00', '11.00']

['9.00', '11.00', '11.00', '9.00', '9.00', '9.00', '9.00', '9.00', '11.00']

['9.00', '9.00']

[' 40.00 ', ' 40.00 ', ' 40.00 ', ' 40.00 ', ' 40.00 ', ' 40.00 ', ' 40.00 ', ' 40.00 ', ' 40.00 ']

[' 5.20', ' 5.00', ' 5.00', ' 2.00', ' 5.00', ' 2.00', ' 2.00', ' 2.00', ' 5.00']


In [17]:
# general data
dfEmployee = pd.DataFrame(employee, columns=['name'])
dfPlant = pd.DataFrame(plant, columns=['plant'])

# time recording
dfMonday = pd.DataFrame(monday, columns=['monday'])
dfTuesday = pd.DataFrame(tuesday, columns=['tuesday'])
dfWednesday = pd.DataFrame(wednesday, columns=['wednesday'])
dfThursday = pd.DataFrame(thursday, columns=['thursday'])
dfFriday = pd.DataFrame(friday, columns=['friday'])
dfSaturday = pd.DataFrame(saturday, columns=['saturday'])
dfSunday = pd.DataFrame(sunday, columns=['sunday'])

# resume of time
dfRT = pd.DataFrame(regularTime, columns=['rt'])
dfOT = pd.DataFrame(overTime, columns=['ot'])


table = pd.concat([dfEmployee, dfPlant, dfMonday, dfTuesday, dfWednesday, dfThursday, dfFriday, dfSaturday, dfSunday, dfRT, dfOT],axis=1)

table = table.sort_values(by='name', ignore_index=True)
table = table.fillna(0)

print(table)

              name         plant monday tuesday wednesday thursday friday  \
0   Anthony Casesa     Canastota  12.00   11.00     11.00    11.00      0   
1     Carlos Lucio        Camden   9.00    9.00      9.00     9.00      0   
2   Daniel Tenorio  Williamstown  12.00   12.00      9.00     9.00      0   
3  Eduardo Barboza     Canastota  12.00   11.00     11.00    11.00   9.00   
4       Eric Ojeda        Camden  12.00   12.00      9.00     9.00      0   
5    Fabian Aviles  Williamstown   9.00    9.10      9.10     9.00   9.00   
6     Gabriel Vera     Canastota  12.00   11.00     11.00    11.00      0   
7   Hector Tirrado  Williamstown  12.00   12.00      9.00     9.00      0   
8     Ian Gonzalez        Camden  12.00   12.00      9.00     9.00      0   

   saturday sunday       rt     ot  
0         0   9.00   40.00    5.00  
1         0      0   40.00    5.00  
2         0      0   40.00    2.00  
3         0   9.00   40.00    5.00  
4         0      0   40.00    2.00  
5      

C:\Users\Carlo\AppData\Local\Temp\ipykernel_5748\383780180.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  table = table.fillna(0)


In [18]:
table.to_excel(r'C:\Users\Carlo\OneDrive\Documents\Python\Docs\Payroll WGN New York.xlsx', index=False)